In [1]:
                                                                                    # ATTEMPT TO USE CROSS-VALIDATION
import tensorflow as tf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input,Dense, Subtract, Add, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from keras.constraints import maxnorm
from sklearn.linear_model import Lasso
                                                                                    
real_dem = pd.read_excel('C:/Users/ERIC/Desktop/RNN_Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',0)

# Training dataset
cen_x = real_dem.iloc[:, 3:8].values
cen_y = real_dem.iloc[:, 8:9].values

#Splitting the data into a 80/20 proportion
cen_x_train, cen_x_test, cen_y_train, cen_y_test = train_test_split(cen_x, cen_y, test_size=0.2, random_state=0)

#Standarization of the data
sc = StandardScaler()
cen_x_train = sc.fit_transform(cen_x_train)
cen_x_test = sc.fit_transform(cen_x_test)

# Input from the current day
input_tensor = Input(shape=(cen_x.shape[1],))

hidden_layer = Dense(100, activation = 'relu',kernel_constraint=maxnorm(3),#Make better results with Dropout
                     kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),#To avoid Over-fitting
                     activity_regularizer=regularizers.l1(0.001))(input_tensor) #Regularize the wieghts to have better accuracy
layer_drop = Dropout(0.5)
hidden_layer1 = Dense(50, activation = 'relu',kernel_constraint=maxnorm(3),
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer)
layer_drop2 = Dropout(0.5)
hidden_layer2 = Dense(100, activation = 'relu', kernel_constraint=maxnorm(3),
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer1)
layer_drop3 = Dropout(0.5)
hidden_layer3 = Dense(25, activation = 'relu', kernel_constraint=maxnorm(3), 
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer2)
output_tensor = Dense(1)(hidden_layer3)

model = Model(input_tensor, output_tensor)

model.compile(optimizer= 'adam',  loss='mean_absolute_error', metrics = ['mean_absolute_error'])

model.summary()

model.fit(cen_x_train,
          cen_y_train,
          epochs=100,
          batch_size=3,
          validation_split=0.1,
          verbose=True)


Using TensorFlow backend.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 100)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 25)                2525      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 26        
Total params: 13,301
Trainable params: 13,301
Non-trainable params: 0
_________________________________________________________

Epoch 41/100
259/259 [==============================] - 0s 891us/sample - loss: 43.2526 - mean_absolute_error: 20.4290 - val_loss: 33.9112 - val_mean_absolute_error: 11.6588
Epoch 42/100
259/259 [==============================] - 0s 907us/sample - loss: 47.5116 - mean_absolute_error: 24.7034 - val_loss: 88.2697 - val_mean_absolute_error: 65.8872
Epoch 43/100
259/259 [==============================] - 0s 884us/sample - loss: 70.3630 - mean_absolute_error: 47.5687 - val_loss: 43.5628 - val_mean_absolute_error: 21.3714
Epoch 44/100
259/259 [==============================] - 0s 880us/sample - loss: 50.4534 - mean_absolute_error: 27.6733 - val_loss: 59.9729 - val_mean_absolute_error: 37.7403
Epoch 45/100
259/259 [==============================] - 0s 900us/sample - loss: 56.5153 - mean_absolute_error: 33.7619 - val_loss: 33.0793 - val_mean_absolute_error: 10.8626
Epoch 46/100
259/259 [==============================] - 0s 872us/sample - loss: 55.2680 - mean_absolute_error: 32.5180 - val_loss:

259/259 [==============================] - 0s 984us/sample - loss: 55.7949 - mean_absolute_error: 33.5557 - val_loss: 54.4916 - val_mean_absolute_error: 32.7488
Epoch 89/100
259/259 [==============================] - 0s 864us/sample - loss: 45.0651 - mean_absolute_error: 22.8379 - val_loss: 45.8728 - val_mean_absolute_error: 24.1653
Epoch 90/100
259/259 [==============================] - 0s 876us/sample - loss: 54.5148 - mean_absolute_error: 32.2858 - val_loss: 46.4625 - val_mean_absolute_error: 24.8180
Epoch 91/100
259/259 [==============================] - 0s 895us/sample - loss: 52.5859 - mean_absolute_error: 30.3812 - val_loss: 31.8966 - val_mean_absolute_error: 10.2314
Epoch 92/100
259/259 [==============================] - 0s 915us/sample - loss: 46.5339 - mean_absolute_error: 24.3480 - val_loss: 41.8744 - val_mean_absolute_error: 20.1902
Epoch 93/100
259/259 [==============================] - 0s 904us/sample - loss: 37.6903 - mean_absolute_error: 15.5159 - val_loss: 54.2181 - va

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(cen_x_train)
print(np.sqrt(mean_squared_error(cen_y_train,pred_train)))
pred = model.predict(cen_x_test)
print(np.sqrt(mean_squared_error(cen_y_test,pred)))

pres_train = pd.DataFrame(pred_train)
pres_train.columns = ['Prediction']
cen_y_train = pd.DataFrame(cen_y_train)
cen_y_train.columns = ['Real Value']
results = pd.concat([pres_train, cen_y_train], axis = 1)
results

31.420798425009803
141.43052635068312


,Prediction,Real Value
0,7410.145020,7474.33949
1,5909.395508,5895.00295
2,5369.213867,5324.94201
3,7694.757324,7739.26681
4,7745.736816,7747.53124
...,...,...
283,6106.110352,6105.05688
284,6121.754395,6075.02433
285,7377.669922,7421.35666
286,6676.027832,6662.96869


In [10]:
names = real_dem.drop('Estimacion de Demanda por Balance (MWh)', axis=1).columns
lasso = Lasso(alpha = 0.1)
lasso_coef = lasso.fit(cen_x, cen_y).coef_
plt.plot(range(len(cen_x)),lasso_coef)
plt.xticks(range(len(cen_x)), cen_x, rotation = 60)
plt.ylabel('Coefficients')
plt.show

KeyError: "['Estimacion de Demanda por Balance (MWh)'] not found in axis"